In [1]:
import pandas as pd

In [6]:
df_raw = pd.read_excel('data/Teeltplan Wijkergouw.xlsx', sheet_name='Gewassen')

/Users/svenvanderburg/miniforge3/envs/teaching/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [7]:
df_raw.head()

,Gewas,Ras,Aantal bedden,Bed\nnummer,Z / PL / VZ,Combinatieteelt,Afstand tussen rij,Afstand in rij,Aantal rijen,Planten per bed,...,Plantdatum,Verwachte oogst (week),Startdatum oogst,Eindweek\noogst,Einddatum oogst,Dagen tot volgroeiing,Dagen de tuin,Dagen in opkweek,Leverancier,Opmerkingen
0,Peulen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tuinboon,ratio,2.0,NaN,Z,inkarnaatklaver,60.0,10.0,2.0,400.000000,...,NaN,24.0,NaN,27.0,NaN,NaN,NaN,NaN,Bingenheimer,NaN
2,Doperwt,Kleine rheinlanderin,2.0,NaN,Z,NaN,45.0,3.0,2.0,1333.333333,...,NaN,20.0,NaN,26.0,NaN,NaN,NaN,NaN,Bingenheimer,NaN
3,Peultje,Ambassador,2.0,NaN,Z,NaN,45.0,3.0,2.0,1333.333333,...,NaN,20.0,NaN,26.0,NaN,NaN,NaN,NaN,Bingenheimer,NaN
4,Kapucijner,blauwschokker,2.0,NaN,Z,NaN,45.0,3.0,2.0,1333.333333,...,NaN,20.0,NaN,26.0,NaN,NaN,NaN,NaN,Bingenheimer,NaN


In [8]:
df_raw.columns

Index(['Gewas', 'Ras', 'Aantal bedden', 'Bed\nnummer', 'Z / PL / VZ',
       'Combinatieteelt', 'Afstand tussen rij', 'Afstand in rij',
       'Aantal rijen', 'Planten per bed', 'Planten nodig', 'Zaaien (week)',
       'Zaaidatum', 'Planten (week)', 'Plantdatum', 'Verwachte oogst (week)',
       'Startdatum oogst', 'Eindweek\noogst', 'Einddatum oogst',
       'Dagen tot volgroeiing', 'Dagen de tuin', 'Dagen in opkweek',
       'Leverancier', 'Opmerkingen'],
      dtype='object')

In [11]:
def get_unique_bed_numbers(df):
    cleaned = df['Bed\nnummer'].str.replace(" ","").str.split(',').dropna()
    return {e for l in cleaned for e in l}

data = list()
mapper = {
    'zaaien': 'Zaaien (week)',
    'planten': 'Planten (week)'} 
for week in range(2, 53):
    for activity in mapper.keys():
        selection = df_raw[df_raw[mapper[activity]] == week]
        for gewas in set(selection['Gewas']):
            this_rows = selection[selection['Gewas'] == gewas]
            # bed_numbers = get_unique_bed_numbers(this_rows)
            # if bed_numbers and next(iter(bed_numbers)) == '?':
            #     print(f'skipping {gewas} becaue of no bednumbers')
            #     continue
            if len(this_rows['Z / PL / VZ'].unique()) > 2:
                raise ValueError(f'Multiple methods for {gewas}')
            method = this_rows['Z / PL / VZ'].unique()[0]
            if activity in ['zaaien', 'planten'] and method == 'DZ':
                to_do = 'direct zaaien'
            elif activity == 'zaaien' and method == 'TP':
                to_do = 'voorzaaien'
                bed_numbers = []
            else:
                to_do = activity

                
            data.append({'Week': week,
                         'Wat doen?': f'{gewas} {to_do}'})
in_time_df = pd.DataFrame(data)

In [12]:
in_time_df.head()

,Week,Wat doen?
0,10,Gele ui plant planten
1,10,Rode ui plant planten
2,10,Sjalot plant planten
3,11,Raapstelen zaaien
4,12,Wortel (zomer) zaaien


In [13]:
in_time_df.to_excel('data/planning-per-week.xlsx', index=False)